## Cut inflation of Triangle Compatibility Test

In [ ]:
import gurobipy as gp
import numpy as np

In [2]:
# from collections import defaultdict
# P_ABC = defaultdict(lambda: 0, {(0,0,1): 1/3, 
#                                 (0,1,0): 1/3, 
#                                 (1,0,0): 1/3})
# P_ABC["ekvhfbjns"]

In [3]:
P_ABC = {(0,0,0): 0,
         (0,0,1): 1/3,
         (0,1,0): 1/3,
         (1,0,0): 1/3,
         (0,1,1): 0,
         (1,0,1): 0,
         (1,1,0): 0,
         (1,1,1): 0}

def P_to_Q(P_ABC):
    """
    given P_ABC calculate Q_AC, Q_AB, Q_BC from P_ABC and the following:
    Q(A,B) = P(A,B)
    Q(A,C) = P(A,C)
    Q(B,C) != P(B,C) 
    Q(B,C) = Q(B)Q(C) = P(B)P(C)
    """
    
    Q_AC = np.array([[sum([P_ABC[a,b,c] for b in range(2)]) for c in range(2)] for a in range(2)])
    Q_AB = Q_AC
    P_B = np.array([sum([P_ABC[a,b,c] for a,c in np.ndindex(2,2)]) for b in range(2)])
    P_C = np.array([sum([P_ABC[a,b,c] for a,b in np.ndindex(2,2)]) for c in range(2)])
    Q_BC = np.array([[P_B[b]*P_C[c] for c in range(2)] for b in range(2)])

    return Q_AC, Q_AB, Q_BC

In [4]:
m = gp.Model()
Q_ABC = m.addMVar((2, 2, 2), name="Q_ABC", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [5]:
Q_AC, Q_AB, Q_BC = P_to_Q(P_ABC)

for a, c in np.ndindex(2, 2):
        m.addConstr(Q_ABC[a, 0, c] + Q_ABC[a, 1, c] == Q_AC[a,c])
        # m.addConstr(sum([Q_ABC[a, b, c] for b in range(2)]) == Q_AC[a,c])
for a, b in np.ndindex(2, 2):
        m.addConstr(Q_ABC[a, b, 0] + Q_ABC[a, b, 1] == Q_AB[a,b])
for b, c in np.ndindex(2, 2):   
        m.addConstr(Q_ABC[0, b, c] + Q_ABC[1, b, c] == Q_BC[b,c])

m.addConstr(Q_ABC.sum() == 1)

<MConstr () *awaiting model update*>

In [6]:
m.update()
m.optimize()

# check if feasible, if so get the solution
if m.status == gp.GRB.OPTIMAL:
    for i, j, k in np.ndindex(2, 2, 2):
        print(f"Q_ABC[{i}, {j}, {k}] = {Q_ABC[i, j, k].X}")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 8 columns and 32 nonzeros
Model fingerprint: 0x8996e5ff
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 13 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Q_ABC[0, 0, 0] = 0.1111111111111111
Q_ABC[0, 0, 1] = 0.2222222222222222
Q_ABC[0, 1, 0] = 0.2222222222222222
Q_ABC[0, 1, 1] = 0.1111111111111111
Q_ABC[1, 0, 0] = 0.333333333

In [7]:
sum([Q_ABC[a,b,c].X for a,b,c in np.ndindex(2,2,2)])

1.0